In [ ]:
import pandas as pd

from estival.wrappers.nevergrad import optimize_model

from tb_incubator.plotting import plot_model_vs_actual, display_plot
from tb_incubator.input import load_targets, load_param_info
from tb_incubator.calibrate import get_bcm

from multiprocessing import cpu_count
pd.options.plotting.backend = "plotly"

In [ ]:
fixed_params = load_param_info()["value"]
#params = {
#    "screening_scaleup_shape": 0.05,
#    "screening_inflection_time": 2006.0
#}
#fixed_params.update(params)
all_targets = load_targets()
file_prefix = "opt3"
covid_effects = {
    'detection_reduction':True,
    #'post_covid_improvement': True
}
bcm = get_bcm(fixed_params, xpert_improvement=True, covid_effects=covid_effects)

In [ ]:
def calibrate_model_with_optimisation(bcm):
    """
    This function performs a model calibration using optimisation. 
    Calibration is performed using the estival package, which implements a wrapper for optimisation methods provided by the nevergrad package. 

    Args:
        bcm: the calibration model object (type BayesianCompartmentalModel) 
    """

    from nevergrad.optimization.differentialevolution import TwoPointsDE
    orunner = optimize_model(bcm, opt_class=TwoPointsDE, num_workers=cpu_count(), budget=4000)
    rec = orunner.minimize(4000)
    optimised_params = rec.value[1]    
   
    return optimised_params

In [ ]:
optimised_params = calibrate_model_with_optimisation(bcm)
optimised_params

In [ ]:
# run the modle with the optimised parameter set
res = bcm.run(optimised_params)
outs = res.derived_outputs

### Results

In [ ]:
fig = plot_model_vs_actual(
    outs, all_targets['notification'], "notification", "Notification", "Modelled vs Data", "Actual data"
)
fig.update_xaxes(range=[1950, 2035])


In [ ]:
fig = plot_model_vs_actual(
    outs, all_targets['prevalence'], "prevalence", "Prevalence", "Modelled vs Data", "Actual data"
)
fig.update_xaxes(range=[1970, 2035])